In [1]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [3]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions can be down, stay, up
        self.action_space = Discrete(3)
        #Temperature array
        self.observation_space = Box(low=np.array([0]), high = np.array([100]))
        #set start temp
        self.state = 38 + random.randint(-3,3)
        #set shower length
        self.shower_length = 60
        
    def step(self, action):
            # Apply action
            # 0 - 1 = -1 
            # 1 - 1 = 0
            # 2 - 1 = 1
            self.state += action - 1
            #reduce shower length by 1 second
            self.shower_length -=1
            
            #calculate reward
            if self.state >= 37 and self.state <=39:
                reward = 1
            else:
                reward = -1
                
            #check if shower is done
            if self.shower_length <=0:
                done = True
            else:
                done = False
                
            #Apply temperature noise
            self.state += random.randint(-1,1)
            #set placeholder for info
            info = {}
            
            #return step information
            return self.state, reward, done, info
    
    def reset(self):
        self.state = 38 + random.randint(-3,3)
        self.shower_length = 60
        return self.state

In [4]:
env = ShowerEnv()

D:\Program_Installation\anaconda\lib\site-packages\gym\spaces\box.py:112: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [7]:
env.action_space.sample()

2

In [9]:
env.observation_space.sample()

array([1.9725913], dtype=float32)

In [19]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print(f'Episode: {episode} score:{score}')

Episode: 1 score:-50
Episode: 2 score:-60
Episode: 3 score:-44
Episode: 4 score:-60
Episode: 5 score:0
Episode: 6 score:-28
Episode: 7 score:-30
Episode: 8 score:-6
Episode: 9 score:-16
Episode: 10 score:-50


## Create a Deep learning model with keras

In [21]:
import numpy as np
import tensorflow
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [22]:
states = env.observation_space.shape
actions = env.action_space.n

In [23]:
def build_model(states, actions):
    model = tensorflow.keras.Sequential()
    model.add(Dense(24, activation = 'relu', input_shape= states))
    model.add(Dense(24, activation= 'relu'))
    model.add(Dense(actions,activation='linear'))
    return model

In [30]:
model = build_model(states,actions)

In [31]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 24)                48        
                                                                 
 dense_4 (Dense)             (None, 24)                600       
                                                                 
 dense_5 (Dense)             (None, 3)                 75        
                                                                 
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


## Build Agent with Keras-RL

In [32]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [33]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy= policy,
                  nb_actions=actions, nb_steps_warmup=10,
                  target_model_update=1e-2)
    return dqn

In [34]:
dqn = build_agent(model,actions)
dqn.compile(Adam(lr=1e-2),metrics=['mae'])
dqn.fit(env,nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)


D:\Program_Installation\anaconda\lib\site-packages\keras\engine\training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


    1/10000 [..............................] - ETA: 44:13 - reward: -1.0000

D:\Program_Installation\anaconda\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
D:\Program_Installation\anaconda\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 10 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)


   47/10000 [..............................] - ETA: 6:27 - reward: -1.0000

D:\Program_Installation\anaconda\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 11 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
D:\Program_Installation\anaconda\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 12 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
D:\Program_Installation\anaconda\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 13 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
D:\Program_Installation\anaconda\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 14 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
D:\Program_Installation\anaconda\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is dep

10000/10000 [==============================] - 45s 4ms/step - reward: 0.1836TA - ETA: 38s - rewar - ETA: 37s - reward: -0 -  - ETA: 34s - reward: -0. - ETA: 34s - rewar - ETA: 33s  - ETA: 32s - reward - ETA: 31s - rewar - ETA: 31s - reward: -0.41 -  - ETA: 27s  - ETA: 26s - reward: - ETA: 25s - rewar - ETA: 25s - reward:  - ETA: 24s - reward: -0.243 - ETA: 24s - reward: -0 - ETA: 24s - reward: -0.235 - ETA: 24s - r - ETA: 1 - ETA: 15s - reward: 3. - ETA: 5s - rewa - ETA: 5s - reward: 0.1 - ETA: 4s  - ETA: 3s - reward: 0.16 - ETA: 3s - reward:   - ETA: 2s - reward:  - ETA: 1s - reward: - ETA: 1s - reward: 0.1 -
166 episodes - episode_reward: 10.928 [-60.000, 58.000] - loss: 1.786 - mae: 6.487 - mean_q: 5.250

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 44s 4ms/step - reward: 0.3680: 0s 
167 episodes - episode_reward: 22.108 [-22.000, 56.000] - loss: 8.654 - mae: 20.780 - mean_q: 31.435

Interval 3 (20000 steps performed)
10000/10000 [===============

In [37]:
scores = dqn.test(env,nb_episodes=100, visualize = False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 60.000, steps: 60
Episode 2: reward: 60.000, steps: 60
Episode 3: reward: 60.000, steps: 60
Episode 4: reward: 60.000, steps: 60
Episode 5: reward: 60.000, steps: 60
Episode 6: reward: 60.000, steps: 60
Episode 7: reward: 60.000, steps: 60
Episode 8: reward: 54.000, steps: 60
Episode 9: reward: 60.000, steps: 60
Episode 10: reward: 60.000, steps: 60
Episode 11: reward: 58.000, steps: 60
Episode 12: reward: 60.000, steps: 60
Episode 13: reward: 60.000, steps: 60
Episode 14: reward: 60.000, steps: 60
Episode 15: reward: 60.000, steps: 60
Episode 16: reward: 60.000, steps: 60
Episode 17: reward: 58.000, steps: 60
Episode 18: reward: 60.000, steps: 60
Episode 19: reward: 60.000, steps: 60
Episode 20: reward: 58.000, steps: 60
Episode 21: reward: 60.000, steps: 60
Episode 22: reward: 60.000, steps: 60
Episode 23: reward: 60.000, steps: 60
Episode 24: reward: 60.000, steps: 60
Episode 25: reward: 58.000, steps: 60
Episode 26: reward: 60.000, st

In [38]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)